In [7]:
import cv2
import numpy as np
import os
import csv

def rgb_to_hsi(image):
    # Normalize pixel values to the range [0, 1]
    image_normalized = image.astype(np.float32) / 255.0

    # Extract R, G, B components
    R, G, B = image_normalized[:, :, 0], image_normalized[:, :, 1], image_normalized[:, :, 2]

    # Compute Intensity (I)
    I = (R + G + B) / 3.0

    # Compute Saturation (S)
    minimum = np.minimum(np.minimum(R, G), B)
    S = 1 - (3 / (R + G + B + 0.001) * minimum)

    # Compute Hue (H)
    numerator = 0.5 * ((R - G) + (R - B))
    denominator = np.sqrt((R - G)**2 + (R - B) * (G - B))
    theta = np.arccos(np.clip(numerator / (denominator + 1e-10), -1.0, 1.0))
    H = np.degrees(theta)
    H[B > G] = 360 - H[B > G]

    return H, S, I


def calculate_entropy(intensity_channel):
    # Calculate histogram of intensity values
    hist, _ = np.histogram(intensity_channel, bins=256, range=(0, 1))

    # Compute probability distribution
    prob_distribution = hist / np.sum(hist)

    # Remove zero probabilities to avoid NaN in the entropy calculation
    non_zero_probs = prob_distribution[prob_distribution > 0]

    # Calculate entropy
    entropy = -np.sum(non_zero_probs * np.log2(non_zero_probs))

    return entropy

def calculate_local_entropy(intensity_channel, window_size=3):
    half_window = window_size // 2

    # Pad the intensity_channel to handle borders
    intensity_channel_padded = np.pad(intensity_channel, ((half_window, half_window), (half_window, half_window)), mode='constant')

    # Calculate histogram of the entire intensity channel
    hist, _ = np.histogram(intensity_channel, bins=256, range=(0, 1))

    # Compute probability distribution
    prob_distribution = hist / np.sum(hist)

    local_entropy = np.zeros_like(intensity_channel)

    for i in range(intensity_channel.shape[0]):
        for j in range(intensity_channel.shape[1]):
            local_region = intensity_channel_padded[i:i + window_size, j:j + window_size]
            local_hist, _ = np.histogram(local_region, bins=256, range=(0, 1))
            local_prob_distribution = local_hist / np.sum(local_hist)

            # Remove zero probabilities to avoid NaN in the entropy calculation
            non_zero_probs = local_prob_distribution[local_prob_distribution > 0]

            # Calculate entropy
            local_entropy[i, j] = -np.sum(non_zero_probs * np.log2(non_zero_probs))

    return local_entropy

def calculate_rms_contrast(intensity_channel):
    # Calculate the standard deviation of the intensity channel
    std_intensity = np.std(intensity_channel)

    return std_intensity

#extra constrast function
def rms_contrast(intensity_channel):
       
    # Calculate RMS contrast
    contrast = np.sqrt(np.mean(intensity_channel**2))
    
    return contrast

def calculate_local_contrast(intensity_channel, window_size=3):
    half_window = window_size // 2

    # Pad the intensity_channel to handle borders
    intensity_channel_padded = np.pad(intensity_channel, ((half_window, half_window), (half_window, half_window)), mode='constant')

    local_contrast = np.zeros_like(intensity_channel)

    for i in range(intensity_channel.shape[0]):
        for j in range(intensity_channel.shape[1]):
            local_region = intensity_channel_padded[i:i + window_size, j:j + window_size]

            # Calculate standard deviation for local contrast
            local_contrast[i, j] = np.std(local_region)

    return local_contrast


def process_image(image_path):
    # Load the input image
    image = cv2.imread(image_path)

    # Convert BGR to RGB (OpenCV loads images in BGR format)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Convert RGB to HSI
    H, S, I = rgb_to_hsi(image_rgb)

    # Calculate RMS contrast using the Intensity (I) component
    rms_contrast_value = calculate_rms_contrast(I)

    #local contrast
    local_contrast_value = calculate_local_contrast(I)
    
    #local entropy
    local_entropy_value = calculate_entropy_contrast(I)
    
    # Calculate the mean value of the S component
    mean_saturation = np.mean(S)

    # Calculate entropy based on the Intensity (I) component
    entropy_I = calculate_entropy(I)

    return [image_path, rms_contrast_value, mean_saturation, entropy_I,local_entropy_value,local_contrast_value]

if __name__ == "__main__":
    input_folder = "pics"  # Replace with the path to your input images folder
    output_csv = "output_results.csv"

    # Limit the number of images for debugging (change this to -1 to process all images)
    num_images_to_process = 5

    # Initialize a CSV file for writing
    with open(output_csv, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        
        # Write the header row
        csv_writer.writerow(["Image Path", "RMS Contrast", "Mean Saturation", "Entropy","Local Entropy","Local Contrast"])

        # Iterate through all image files in the folder
        processed_images_count = 0
        for filename in os.listdir(input_folder):
            if filename.endswith((".jpg", ".png")):
                print("Processing:", filename)  # Print the filename being processed

                # Write the results to the CSV file
                csv_writer.writerow(results)
                
                # Increment the processed images count
                processed_images_count += 1

                # Break the loop if we've processed the desired number of images
                if num_images_to_process > 0 and processed_images_count >= num_images_to_process:
                    break

    print(f"Processed {processed_images_count} images and saved results in {output_csv}")


Processing: pexels-aron-visuals-1743165.jpg
Processing: pexels-eberhard-grossgasteiger-1366919.jpg
Processing: pexels-james-wheeler-1486974.jpg
Processing: pexels-pixabay-147411.jpg
Processing: pexels-ricky-esquivel-1586298.jpg
Processed 5 images and saved results in output_results.csv


In [ ]:
import cv2
import numpy as np
import os
import csv

def rgb_to_hsi(image):
    # Normalize pixel values to the range [0, 1]
    image_normalized = image.astype(np.float32) / 255.0

    # Extract R, G, B components
    R, G, B = image_normalized[:, :, 0], image_normalized[:, :, 1], image_normalized[:, :, 2]

    # Compute Intensity (I)
    I = (R + G + B) / 3.0

    # Compute Saturation (S)
    minimum = np.minimum(np.minimum(R, G), B)
    S = 1 - (3 / (R + G + B + 0.001) * minimum)

    # Compute Hue (H)
    numerator = 0.5 * ((R - G) + (R - B))
    denominator = np.sqrt((R - G)**2 + (R - B) * (G - B))
    theta = np.arccos(np.clip(numerator / (denominator + 1e-10), -1.0, 1.0))
    H = np.degrees(theta)
    H[B > G] = 360 - H[B > G]

    return H, S, I


def calculate_entropy(intensity_channel):
    # Calculate histogram of intensity values
    hist, _ = np.histogram(intensity_channel, bins=256, range=(0, 1))

    # Compute probability distribution
    prob_distribution = hist / np.sum(hist)

    # Remove zero probabilities to avoid NaN in the entropy calculation
    non_zero_probs = prob_distribution[prob_distribution > 0]

    # Calculate entropy
    entropy = -np.sum(non_zero_probs * np.log2(non_zero_probs))

    return entropy

def calculate_local_entropy(intensity_channel, window_size=3):
    half_window = window_size // 2

    # Pad the intensity_channel to handle borders
    intensity_channel_padded = np.pad(intensity_channel, ((half_window, half_window), (half_window, half_window)), mode='constant')

    # Calculate histogram of the entire intensity channel
    hist, _ = np.histogram(intensity_channel, bins=256, range=(0, 1))

    # Compute probability distribution
    prob_distribution = hist / np.sum(hist)

    local_entropy = np.zeros_like(intensity_channel)

    for i in range(intensity_channel.shape[0]):
        for j in range(intensity_channel.shape[1]):
            local_region = intensity_channel_padded[i:i + window_size, j:j + window_size]
            local_hist, _ = np.histogram(local_region, bins=256, range=(0, 1))
            local_prob_distribution = local_hist / np.sum(local_hist)

            # Remove zero probabilities to avoid NaN in the entropy calculation
            non_zero_probs = local_prob_distribution[local_prob_distribution > 0]

            # Calculate entropy
            local_entropy[i, j] = -np.sum(non_zero_probs * np.log2(non_zero_probs))

    return local_entropy

def calculate_rms_contrast(intensity_channel):
    # Calculate the standard deviation of the intensity channel
    std_intensity = np.std(intensity_channel)

    return std_intensity

#extra constrast function
def rms_contrast(intensity_channel):
       
    # Calculate RMS contrast
    contrast = np.sqrt(np.mean(intensity_channel**2))
    
    return contrast

def calculate_local_contrast(intensity_channel, window_size=3):
    half_window = window_size // 2

    # Pad the intensity_channel to handle borders
    intensity_channel_padded = np.pad(intensity_channel, ((half_window, half_window), (half_window, half_window)), mode='constant')

    local_contrast = np.zeros_like(intensity_channel)

    for i in range(intensity_channel.shape[0]):
        for j in range(intensity_channel.shape[1]):
            local_region = intensity_channel_padded[i:i + window_size, j:j + window_size]

            # Calculate standard deviation for local contrast
            local_contrast[i, j] = np.std(local_region)

    return local_contrast

def calculate_entropy_contrast(intensity_channel, window_size=3):
    local_entropy = calculate_local_entropy(intensity_channel, window_size)
    local_contrast = calculate_local_contrast(intensity_channel, window_size)
    return local_entropy, local_contrast

# Modify the process_image function
def process_image(image_path):
    # Load the input image
    image = cv2.imread(image_path)

    # Convert BGR to RGB (OpenCV loads images in BGR format)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Convert RGB to HSI
    H, S, I = rgb_to_hsi(image_rgb)

    # Calculate RMS contrast using the Intensity (I) component
    rms_contrast_value = calculate_rms_contrast(I)

    # Calculate local entropy and local contrast
    local_entropy_value, local_contrast_value = calculate_entropy_contrast(I)

    # Calculate the mean value of the S component
    mean_saturation = np.mean(S)

    # Calculate entropy based on the Intensity (I) component
    entropy_I = calculate_entropy(I)

    return [image_path, rms_contrast_value, mean_saturation, entropy_I, local_entropy_value, local_contrast_value]

if __name__ == "__main__":
    input_folder = "pics"  # Replace with the path to your input images folder
    output_csv = "output_results.csv"

    # Limit the number of images for debugging (change this to -1 to process all images)
    num_images_to_process = 5

    # Initialize a CSV file for writing
    with open(output_csv, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        
        # Write the header row
        csv_writer.writerow(["Image Path", "RMS Contrast", "Mean Saturation", "Entropy", "Local Entropy", "Local Contrast"])

        # Iterate through all image files in the folder
        processed_images_count = 0
        for filename in os.listdir(input_folder):
            if filename.endswith((".jpg", ".png")):
                print("Processing:", filename)  # Print the filename being processed

                # Process the image and get the results
                results = process_image(os.path.join(input_folder, filename))

                # Write the results to the CSV file
                csv_writer.writerow(results)
                
                # Increment the processed images count
                processed_images_count += 1

                # Break the loop if we've processed the desired number of images
                if num_images_to_process > 0 and processed_images_count >= num_images_to_process:
                    break

    print(f"Processed {processed_images_count} images and saved results in {output_csv}")


Processing: pexels-aron-visuals-1743165.jpg
